# Treinamento de CNN do Zero - Parte 1: Preparação

Este notebook implementa a primeira parte do treinamento de uma Rede Neural Convolucional (CNN) do zero para classificar imagens nas mesmas categorias que usamos nos modelos YOLO. Diferentemente do YOLO, que é um modelo de detecção de objetos, a CNN que vamos treinar é um modelo de classificação de imagens.

Nesta primeira parte, vamos focar na preparação dos dados e na definição da arquitetura da CNN.

## 1. Configuração do Ambiente

Primeiro, vamos importar as bibliotecas necessárias e configurar o ambiente.

In [ ]:
# Verificar se o ambiente já foi configurado
import os
import sys

# Se o ambiente ainda não foi configurado, execute o setup_env.sh
if not os.path.exists('../yolov5'):
    print("Configurando o ambiente com setup_env.sh...")
    !chmod +x ../setup_env.sh
    !../setup_env.sh
else:
    print("Ambiente já configurado.")

# Importar bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets
import yaml
import time
import random
from pathlib import Path
from tqdm.notebook import tqdm

## 2. Preparação dos Dados

Vamos preparar os dados para treinamento da CNN. Como a CNN é um modelo de classificação, precisamos adaptar nosso dataset de detecção de objetos para classificação.

In [ ]:
# Carregar o arquivo data.yaml para obter as categorias
if os.path.exists('../data/data.yaml'):
    with open('../data/data.yaml', 'r') as f:
        data_yaml = yaml.safe_load(f)
    categories = data_yaml['names']
    print(f"Categorias: {categories}")
else:
    print("Arquivo data.yaml não encontrado. Usando categorias padrão.")
    categories = ['apple', 'banana']

# Definir diretórios de dados
train_dir = '../dataset/train/images'
val_dir = '../dataset/val/images'
test_dir = '../dataset/test/images'

# Verificar se os diretórios existem
for dir_path in [train_dir, val_dir, test_dir]:
    if not os.path.exists(dir_path):
        print(f"❌ Diretório não encontrado: {dir_path}")
    else:
        print(f"✅ Diretório encontrado: {dir_path}")
        print(f"   Número de imagens: {len([f for f in os.listdir(dir_path) if f.endswith(('.jpg', '.jpeg', '.png', '.avif'))])}")

### 2.1 Criação de Dataset e DataLoader

Vamos criar classes personalizadas para carregar e pré-processar nossos dados.

In [ ]:
# Definir transformações para as imagens
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Classe personalizada para o dataset
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, categories, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.categories = categories
        
        # Listar todas as imagens
        self.img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png', '.avif'))]
        
        # Determinar a classe de cada imagem com base no nome do arquivo
        self.labels = []
        for img_file in self.img_files:
            # Assumindo que o nome do arquivo começa com o nome da categoria
            # Por exemplo: categoria_a_001.jpg -> categoria_a
            for i, category in enumerate(categories):
                if category.lower() in img_file.lower():
                    self.labels.append(i)
                    break
            else:
                # Se não encontrar a categoria no nome do arquivo, usar a primeira categoria
                self.labels.append(0)
    
    def __len__(self):
        return len(self.img_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Criar datasets
train_dataset = CustomImageDataset(train_dir, categories, transform=train_transforms)
val_dataset = CustomImageDataset(val_dir, categories, transform=val_test_transforms)
test_dataset = CustomImageDataset(test_dir, categories, transform=val_test_transforms)

# Criar dataloaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Verificar os datasets
print(f"Tamanho do dataset de treino: {len(train_dataset)}")
print(f"Tamanho do dataset de validação: {len(val_dataset)}")
print(f"Tamanho do dataset de teste: {len(test_dataset)}")

# Verificar a distribuição das classes
train_labels = train_dataset.labels
val_labels = val_dataset.labels
test_labels = test_dataset.labels

print("\nDistribuição das classes:")
print("Treino:")
for i, category in enumerate(categories):
    count = train_labels.count(i)
    print(f"  - {category}: {count} ({count/len(train_labels)*100:.1f}%)")

print("\nValidação:")
for i, category in enumerate(categories):
    count = val_labels.count(i)
    print(f"  - {category}: {count} ({count/len(val_labels)*100:.1f}%)")

print("\nTeste:")
for i, category in enumerate(categories):
    count = test_labels.count(i)
    print(f"  - {category}: {count} ({count/len(test_labels)*100:.1f}%)")

### 2.2 Visualização de Algumas Imagens

Vamos visualizar algumas imagens do dataset para verificar se estão sendo carregadas corretamente.

In [ ]:
# Função para desnormalizar imagens
def denormalize(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    return tensor * std + mean

# Obter algumas imagens do dataset de treino
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Visualizar as imagens
plt.figure(figsize=(15, 8))
for i in range(min(8, len(images))):
    plt.subplot(2, 4, i+1)
    img = denormalize(images[i])
    img = img.permute(1, 2, 0).numpy()
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.title(f"{categories[labels[i]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

## 3. Definição da Arquitetura da CNN

Vamos definir a arquitetura da nossa CNN do zero.

In [ ]:
# Definir a arquitetura da CNN
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        
        # Camadas convolucionais
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        
        # Camadas de pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Camadas de batch normalization
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        
        # Camadas fully connected
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # Bloco 1
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        
        # Bloco 2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        
        # Bloco 3
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        # Bloco 4
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Criar o modelo
model = CustomCNN(num_classes=len(categories))
print(model)

# Verificar o número de parâmetros
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal de parâmetros: {total_params:,}")
print(f"Parâmetros treináveis: {trainable_params:,}")

## 4. Salvando os Dados Preparados

Vamos salvar os dados preparados para uso na segunda parte do notebook.

In [ ]:
# Criar diretório para salvar os dados
os.makedirs('../models/cnn', exist_ok=True)

# Salvar o modelo não treinado
torch.save(model.state_dict(), '../models/cnn/cnn_initial.pt')
print("Modelo inicial salvo em '../models/cnn/cnn_initial.pt'")

# Salvar as categorias
with open('../models/cnn/cnn_categories.txt', 'w') as f:
    for category in categories:
        f.write(f"{category}\n")
print("Categorias salvas em '../models/cnn/cnn_categories.txt'")

print("\nTudo pronto para o treinamento na Parte 2!")

## 5. Próximos Passos

Na próxima parte (Parte 2), vamos:
1. Treinar a CNN do zero
2. Avaliar o desempenho do modelo
3. Visualizar algumas predições
4. Comparar o desempenho com os modelos YOLO